# Deep Learning with Python

## 2.1 A first look at a neural network

> 初识神经网络

学编程语言从 “Hello World” 开始，学 Deep learning 从 `MINST` 开始。

MNIST 用来训练手写数字识别， 它包含 28x28 的灰度手写图片，以及每张图片对应的标签(0~9的值)。

### 导入MNIST数据集

In [1]:
# Loading the MNIST dataset in Keras
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

看一下训练集：

In [2]:
print(train_images.shape)
print(train_labels.shape)
train_labels

(60000, 28, 28)
(60000,)


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

这是测试集：

In [3]:
print(test_images.shape)
print(test_labels.shape)
test_labels

(10000, 28, 28)
(10000,)


array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

### 网络构建

我们来构建一个用来学习 MNIST 集的神经网络：

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation='softmax'))

神经网络是一个个「层」组成的。
一个「层」就像是一个“蒸馏过滤器”，它会“过滤”处理输入的数据，从里面“精炼”出需要的信息，然后传到下一层。

这样一系列的「层」组合起来，像流水线一样对数据进行处理。
层层扬弃，让被处理的数据，或者说“数据的表示”对我们最终希望的结果越来越“有用”。

我们刚才这段代码构建的网络包含两个「Dense 层」，这么叫是因为它们是密集连接（densely connected）或者说是 *全连接* 的。

数据到了最后一层（第二层），是一个 **10路** 的 softmax 层。
这个层输出的是一个数组，包含 10 个概率值（它们的和为1），这个输出「表示」的信息就对我们预测图片对应的数字相当有用了。
事实上这输出中的每一个概率值就分别代表输入图片属于10个数字（0～9）中的一个的概率！

### 编译

接下来，我们要 *编译* 这个网络，这个步骤需要给3个参数：

- 损失函数：评价你这网络表现的好不好的函数
- 优化器：怎么更新（优化）你这个网络
- 训练和测试过程中需要监控的指标，比如这个例子里，我们只关心一个指标 —— 预测的精度

In [5]:
network.compile(loss="categorical_crossentropy",
                optimizer='rmsprop',
                metrics=['accuracy'])

### 预处理

#### 图形处理

我们还需要处理一下图形数据，把它变成我们的网络认识的样子。

MNIST 数据集里的图片是 28x28 的，每个值是属于 [0, 255] 的 uint8。
而我们的神经网络想要的是 28x28 的在 [0, 1] 中的 float32。

In [6]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

#### 标签处理

同样，标签也是需要处理一下的。

In [7]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### 训练网络

In [8]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.2549 - accuracy: 0.9254
Epoch 2/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.1025 - accuracy: 0.9693
Epoch 3/5
60000/60000 [==============================] - 2s 35us/sample - loss: 0.0676 - accuracy: 0.9800
Epoch 4/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.0491 - accuracy: 0.9848
Epoch 5/5
60000/60000 [==============================] - 2s 42us/sample - loss: 0.0369 - accuracy: 0.9888


可以看到，训练很快，一会儿就对训练集有 98%+ 的精度了。

再用测试集去试试：

In [9]:
test_loss, test_acc = network.evaluate(test_images, test_labels, verbose=2)    # verbose=2 to avoid a looooong progress bar that fills the screen with '='. https://github.com/tensorflow/tensorflow/issues/32286
print('test_acc:', test_acc)

10000/1 - 0s - loss: 0.0362 - accuracy: 0.9789
test_acc: 0.9789


我们训练好的网络在测试集下的表现并没有之前在训练集中那么好，这是「过拟合」的锅。